In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
spark = SparkSession.builder.appName("Employee_Reporting").getOrCreate()

25/05/08 05:41:21 WARN Utils: Your hostname, TTNPL-kanishkasharma resolves to a loopback address: 127.0.1.1; using 192.168.1.54 instead (on interface wlp0s20f3)
25/05/08 05:41:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/08 05:41:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Append-Only Yearly Tables:
employee_leave_quota_data.csv → Yearly Quota Table

employee_leave_calendar_data.csv → Yearly Holiday Table

KEEP ACTIVE EMPLOYEES ONLY FOR LEAVE DATA 

In [3]:
df_calendar = spark.read.option("header", True).csv("employee_leave_calendar_data.csv") 

calendar_data = df_calendar.withColumn("date", col("date").cast(DateType()))
calendar_data.show()

+------+----------+
|reason|      date|
+------+----------+
|1KDIHR|2023-07-28|
|L52O0S|2023-08-10|
|CFBUWO|2023-08-04|
|EZNHM2|2023-06-22|
|HTI2Z9|2023-07-24|
|LUISBT|2023-03-04|
|E8AF7Z|2023-12-09|
|FJ2A08|2023-06-03|
|4EIDAB|2023-07-16|
|9BJCCY|2023-08-03|
|KM7BZW|2024-04-19|
|OS7EJN|2024-05-11|
|LBRNTP|2024-12-15|
|L095Q3|2024-07-19|
|180M0W|2024-04-26|
|NP47QH|2024-02-28|
|QZDYCX|2024-03-11|
|606QBS|2024-02-21|
|H3AJUH|2024-08-08|
|JK1REQ|2024-01-18|
+------+----------+



In [4]:
df_leave_quota = spark.read.option("header", "true") \
                   .option("inferSchema", "true") \
                   .csv("employee_leave_quota_data.csv")

quota_data = df_leave_quota.withColumn("leave_quota", df_leave_quota["leave_quota"].cast("int")) \
                   .withColumn("year", df_leave_quota["year"].cast("int"))
quota_data.show()

+----------+-----------+----+
|    emp_id|leave_quota|year|
+----------+-----------+----+
|7175690919|         23|2023|
|7175690919|         26|2024|
|3345338467|         24|2023|
|3345338467|         28|2024|
|4047753983|         26|2023|
|4047753983|         24|2024|
|5551916669|         22|2023|
|5551916669|         21|2024|
|6533954001|         28|2023|
|6533954001|         30|2024|
|7170426383|         27|2023|
|7170426383|         23|2024|
|3755067549|         27|2023|
|3755067549|         20|2024|
| 272449155|         20|2023|
| 272449155|         28|2024|
|2903567372|         28|2023|
|2903567372|         28|2024|
|4397504229|         20|2023|
|4397504229|         28|2024|
+----------+-----------+----+
only showing top 20 rows



In [11]:
from pyspark.sql.functions import col, count

quota_data.groupBy("emp_id") \
  .agg(count("*").alias("count")) \
  .filter(col("count") > 2) \
  .select("emp_id") \
  .show()


+----------+
|    emp_id|
+----------+
|7601580536|
|5897722297|
|5466669121|
+----------+



In [5]:
leave_data = spark.read.option("header", "true") \
                   .option("inferSchema", "true") \
                   .csv("employee_leave_data.csv")
# Step 1: Ensure date is in correct format
leave_data = leave_data.withColumn("date", to_date(col("date"), "yyyy-M-d"))
leave_data.show()

[Stage 6:============================================>              (6 + 2) / 8]

+----------+----------+---------+
|    emp_id|      date|   status|
+----------+----------+---------+
|7175690919|2023-01-01|CANCELLED|
|7175690919|2023-12-25|   ACTIVE|
|7175690919|2023-12-05|   ACTIVE|
|7175690919|2023-12-14|   ACTIVE|
|7175690919|2023-12-15|CANCELLED|
|7175690919|2023-07-01|   ACTIVE|
|7175690919|2023-08-05|   ACTIVE|
|7175690919|2023-12-10|   ACTIVE|
|7175690919|2023-03-14|   ACTIVE|
|7175690919|2023-03-22|   ACTIVE|
|7175690919|2023-11-07|   ACTIVE|
|7175690919|2023-04-23|   ACTIVE|
|7175690919|2023-04-16|   ACTIVE|
|7175690919|2023-05-19|   ACTIVE|
|7175690919|2023-05-26|   ACTIVE|
|7175690919|2023-07-18|   ACTIVE|
|7175690919|2024-11-19|   ACTIVE|
|7175690919|2024-05-23|   ACTIVE|
|7175690919|2024-01-01|   ACTIVE|
|7175690919|2024-12-27|   ACTIVE|
+----------+----------+---------+
only showing top 20 rows



For each (emp_id, date):

Count how many times each status occurs: "ACTIVE" and "CANCELLED"

Apply logic:

If "CANCELLED" count > "ACTIVE" count → status is "CANCELLED"

If "ACTIVE" count > "CANCELLED" count → status is "ACTIVE"

If counts are equal → status is "CANCELLED" (since cancel overrides)

In [6]:
# Step 2: Map status to two count columns
leave_status_counts = leave_data.withColumn("is_active", when(col("status") == "ACTIVE", 1).otherwise(0)) \
                                .withColumn("is_cancelled", when(col("status") == "CANCELLED", 1).otherwise(0))
leave_status_counts.show()

+----------+----------+---------+---------+------------+
|    emp_id|      date|   status|is_active|is_cancelled|
+----------+----------+---------+---------+------------+
|7175690919|2023-01-01|CANCELLED|        0|           1|
|7175690919|2023-12-25|   ACTIVE|        1|           0|
|7175690919|2023-12-05|   ACTIVE|        1|           0|
|7175690919|2023-12-14|   ACTIVE|        1|           0|
|7175690919|2023-12-15|CANCELLED|        0|           1|
|7175690919|2023-07-01|   ACTIVE|        1|           0|
|7175690919|2023-08-05|   ACTIVE|        1|           0|
|7175690919|2023-12-10|   ACTIVE|        1|           0|
|7175690919|2023-03-14|   ACTIVE|        1|           0|
|7175690919|2023-03-22|   ACTIVE|        1|           0|
|7175690919|2023-11-07|   ACTIVE|        1|           0|
|7175690919|2023-04-23|   ACTIVE|        1|           0|
|7175690919|2023-04-16|   ACTIVE|        1|           0|
|7175690919|2023-05-19|   ACTIVE|        1|           0|
|7175690919|2023-05-26|   ACTIV

In [7]:
# Step 3: Group by emp_id + date and sum status flags
grouped = leave_status_counts.groupBy("emp_id", "date") \
    .agg(
        sum("is_active").alias("active_count"),
        sum("is_cancelled").alias("cancelled_count")
    )
grouped.show()

25/05/08 05:41:33 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/08 05:41:33 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/08 05:41:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/08 05:41:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/08 05:41:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/08 05:41:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/08 05:41:34 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
[Stage 9:===================================================>       (7 + 1) / 8]

+----------+----------+------------+---------------+
|    emp_id|      date|active_count|cancelled_count|
+----------+----------+------------+---------------+
|7175690919|2023-05-19|           1|              0|
| 272449155|2023-11-05|           1|              0|
|9154542239|2023-06-08|           1|              0|
|9154542239|2023-07-10|           1|              0|
|1260589765|2023-05-16|           1|              0|
|6727837063|2023-02-12|           1|              0|
|9623856554|2024-02-11|           1|              0|
|3368105384|2023-05-10|           1|              0|
|6619102777|2023-06-18|           1|              0|
|6619102777|2023-07-28|           1|              0|
|5568316074|2023-02-03|           1|              0|
| 706688810|2024-07-20|           1|              0|
|4530872653|2024-03-13|           1|              0|
|5033623298|2023-09-21|           1|              0|
|8022137890|2023-04-13|           1|              0|
|5468748102|2024-09-11|           1|          

In [8]:
# Step 4: Apply status logic based on counts
leave_data_updated = grouped.withColumn(
    "status",
    when(col("cancelled_count") >= col("active_count"), "CANCELLED").otherwise("ACTIVE")
).select("emp_id", "date", "status")
leave_data_updated.show()

25/05/08 05:41:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/08 05:41:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/08 05:41:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/08 05:41:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/08 05:41:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/08 05:41:42 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/08 05:41:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
[Stage 12:==================================================>       (7 + 1) / 8]

+----------+----------+------+
|    emp_id|      date|status|
+----------+----------+------+
|7175690919|2023-05-19|ACTIVE|
| 272449155|2023-11-05|ACTIVE|
|9154542239|2023-06-08|ACTIVE|
|9154542239|2023-07-10|ACTIVE|
|1260589765|2023-05-16|ACTIVE|
|6727837063|2023-02-12|ACTIVE|
|9623856554|2024-02-11|ACTIVE|
|3368105384|2023-05-10|ACTIVE|
|6619102777|2023-06-18|ACTIVE|
|6619102777|2023-07-28|ACTIVE|
|5568316074|2023-02-03|ACTIVE|
| 706688810|2024-07-20|ACTIVE|
|4530872653|2024-03-13|ACTIVE|
|5033623298|2023-09-21|ACTIVE|
|8022137890|2023-04-13|ACTIVE|
|5468748102|2024-09-11|ACTIVE|
|8328042768|2023-07-10|ACTIVE|
|8523798777|2024-04-23|ACTIVE|
|3286077338|2024-01-24|ACTIVE|
| 958054928|2024-10-15|ACTIVE|
+----------+----------+------+
only showing top 20 rows



In [9]:
dupes = leave_data_updated.groupBy("emp_id", "date").count().filter("count > 1")
if dupes.count() > 0:
    print("⚠️ Duplicate ACTIVE leave records found:")
    dupes.show()

25/05/08 05:41:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/08 05:41:54 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
                                                                                


Generate a daily table (at 7:00 UTC) showing currently active employees by designation and count.

Active Employees by Designation (Daily @ 7:00 UTC)
Input:

employee_timeframe_data (already processed and includes status)

Logic:

Filter records where status = 'Active'

Group by designation

Count employees


In [10]:
#employee_timeline = spark.read.format("delta").load("s3://your-bucket/employee_timeframe_delta/")

active_employees = df4.filter(col("status") == "Active") \
    .groupBy("designation") \
    .agg(count("emp_id").alias("active_emp_count"))\
    .sort("active_emp_count")
active_employees.show()

#active_employees.write.mode("overwrite").format("delta").save("s3://your-bucket/reports/active_employee_counts")

NameError: name 'df4' is not defined

Potential Leave Abuse > 8% of Working Days (Daily @ 7:00 UTC)
Inputs:

employee_leave_data.csv

employee_leave_calendar_data.csv

Generate working days from tomorrow to end of year

Exclude:

Weekends

Public holidays

Cancelled leaves

Logic:

Calculate remaining working days this year.

Filter employee_leave_data:

status = ACTIVE

leave date in future

not weekend, not holiday

Count leaves per employee

If leave_count > 8% of working days, flag

In [ ]:
from pyspark.sql.functions import (
    col, lit, to_date, dayofweek, explode, sequence, countDistinct, expr, year
)
from pyspark.sql.types import DateType
from datetime import datetime

# ========== Step 1: Setup Dates ==========
# today = datetime.utcnow().date()
today = datetime(2024,4,25) # when i explicitly gave the date
current_date_str = today.strftime('%Y-%m-%d')
end_of_year_str = f"{today.year}-12-31"
run_date_str = today.strftime('%Y-%m-%d')

print("📅 Running for date range:", current_date_str, "to", end_of_year_str)

In [ ]:
# ========== Step 2: Prepare Leave Data ==========
leave_data_final = leave_data_updated.withColumn("date", to_date(col("date"), "yyyy-MM-dd"))

active_leaves = leave_data_final.filter(
    (col("status") == "ACTIVE") &
    (col("date") >= lit(current_date_str))
).dropDuplicates(["emp_id", "date"])

print("🟡 Active leave records:", active_leaves.count())

In [ ]:
# ========== Step 3: Prepare Holiday Data ==========
holidays = calendar_data \
    .withColumn("date", to_date("date")) \
    .filter(col("date") >= lit(current_date_str)) \
    .select("date").distinct() \
    .withColumn("holiday", lit(1))
holidays.show()

In [ ]:
# ========== Step 4: Generate Working Days ==========
date_range = spark.createDataFrame([()]).select(
    explode(sequence(to_date(lit(current_date_str)), to_date(lit(end_of_year_str)))).alias("date")
)

working_days = date_range \
    .withColumn("day_of_week", dayofweek("date")) \
    .filter(~col("day_of_week").isin([1, 7])) \
    .drop("day_of_week") \
    .join(holidays, on="date", how="left_anti") \
    .persist()

total_working_days = working_days.count()
print("📆 Total working days left:", total_working_days)

In [ ]:
if total_working_days == 0:
    print("⚠️ No valid working days found. Exiting early.")
else:
    # ========== Step 5: Join Leaves with Working Days ==========
    leaves_on_working_days = active_leaves.join(working_days, on="date", how="inner") \
        .select("emp_id", "date").distinct()

    print("📌 Leaves on working days:", leaves_on_working_days.count())

In [ ]:
# ========== Step 6: Count Leaves and Flag ==========
emp_leave_counts = leaves_on_working_days.groupBy("emp_id") \
                   .agg(countDistinct("date").alias("leaves_taken"))

flagged = emp_leave_counts.withColumn(
        "leave_percent",
        (col("leaves_taken") / lit(total_working_days)) * 100
    ).withColumn(
        "flagged", expr("CASE WHEN leave_percent > 8 THEN 'Yes' ELSE 'No' END")
    ).withColumn("run_date", lit(run_date_str))

# Show result
flagged.show()

In [ ]:
count_flagged = flagged.filter(col("flagged") == "Yes").count()
print(f"✅ Employees flagged (leave > 8%): {count_flagged}")

In [ ]:
employee_flagged = flagged.filter(col("emp_id") == '154225493')
employee_flagged.show()

Leave Quota Overuse >80% (Monthly on 1st @ 7:00 UTC)
Inputs:

employee_leave_data.csv

employee_leave_quota_data.csv

Logic:

For current year, count leaves where status = ACTIVE

Join with leave quota data

If (leave_taken / quota) > 0.8, flag

Group by manager (placeholder or you can skip the file-per-manager if no manager field is present)


In [ ]:
from dateutil.relativedelta import relativedelta
# ========== Step 1: Setup Reporting Dates ==========
# Simulate: Run on the 1st of any month
#today = datetime.utcnow().date()
today = datetime(2024, 11, 1).date()  # ← change to the 1st of any month you're testing
print(today)
# Calculate end of previous month as cutoff
report_cutoff = today - relativedelta(days=1)  # e.g., 2024-04-30
report_cutoff_str = report_cutoff.strftime("%Y-%m-%d")
run_date_str = today.strftime("%Y-%m-%d")
current_year = today.year

print(f"📊 Reporting for period: Jan 1 to {current_year}")

In [ ]:
# ========== Step 2: Prepare Clean Leave Data ==========
leave_data_updated = leave_data_updated.withColumn("date", to_date("date"))

# Filter ACTIVE leaves in the current year up to the cutoff
valid_leaves = leave_data_updated.filter(
    (col("status") == "ACTIVE") &
    (year(col("date")) == current_year) &
    (col("date") <= lit(report_cutoff_str))
)
valid_leaves.show()

In [ ]:
# Generate date range Jan 1 → report_cutoff
date_range = spark.createDataFrame([()]).select(
    explode(sequence(
        to_date(lit(f"{current_year}-01-01")),
        to_date(lit(report_cutoff_str))
    )).alias("date")
)

# Remove weekends
working_days = date_range.withColumn("dow", dayofweek("date")) \
    .filter(~col("dow").isin([1, 7])) \
    .drop("dow")

working_days.show()

In [ ]:
# Join holiday calendar and remove holiday dates
holidays = calendar_data.withColumn("date", to_date("date")).select("date").distinct()
working_days = working_days.join(holidays, on="date", how="left_anti")
print("total working days", working_days.count())

# Join with valid leave data
leaves_on_working_days = valid_leaves.join(working_days, on="date", how="inner")

leaves_on_working_days.show()

In [ ]:
# Count real working leaves
leaves_taken = leaves_on_working_days.groupBy("emp_id") \
    .agg(countDistinct("date").alias("leaves_taken"))
leaves_taken.show()

In [ ]:
# ========== Step 3: Join with Quota & Flag ==========
leave_usage = leaves_taken.join(quota_data, on="emp_id", how="inner") \
    .filter(col("year") == lit(current_year)) \
    .withColumn("leave_percent", (col("leaves_taken") / col("leave_quota")) * 100) \
    .withColumn("flagged", expr("CASE WHEN leave_percent > 80 THEN 'Yes' ELSE 'No' END")) \
    .filter(col("flagged") == "Yes") \
    .withColumn("run_date", lit(run_date_str))

leave_usage.show()

In [ ]:
employee_leaves = leaves_taken.filter(col("emp_id") == '1226091381')
employee_leaves.show()

In [ ]:
flagged_count = leave_usage.count()
print(f"🔍 Total flagged employees: {flagged_count}")